### Test Model After Training

In [14]:
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")

### 🟢 Cleaning Text Functions

In [15]:
_stopwords = stopwords.words("english")

def _remove_Stopwords(string:str):
    return [x for x in string if x not in _stopwords]

def _Lemmatizong_data(text:list):
    Lemmarizer = WordNetLemmatizer()
    return [Lemmarizer.lemmatize(word=word,pos="v") for word in text]

In [20]:

def CleanText(Text:str):
    Text = Text.lower()
    Text = re.sub(pattern=r"[^\w\s]",repl=" ",string=Text)
    Text = re.sub(pattern=r"\b[a-z]+n\s+t\b",repl=" ",string=Text)
    Text = re.sub(pattern=r"\b[a-z]{1}\b",repl=" ",string=Text)
    Text = word_tokenize(Text)
    Text = _remove_Stopwords(Text)
    Text = _Lemmatizong_data(Text) 
    return " ".join(Text)

### ✅ Models Loading...

In [17]:
model = load_model("sentiment_model.keras")

with open("tokenizer.pickle", "rb") as f:
    tokenizer = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

### ✔ Inference Function

In [ ]:
def ClassifyText(NoteText:str):

    text = [CleanText(Text=NoteText)]
    seq = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(seq, maxlen=25, padding="post")

    y_pred = model.predict(padded)
    label = y_pred.argmax(axis=1)

    label_name = label_encoder.inverse_transform(label)[0]

    return  label_name , y_pred[0][label[0]]

In [22]:
ClassifyText("I'm Very happy because today i finished my exams")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


('Happy', np.float32(0.5736358))

### ✅ Model API Testing With **Hugging Face**

In [23]:
import requests

url = "https://awadallayossef-lstm-sentimant-analysis.hf.space/Classify"
params = {"NoteText": "I'm Very happy because today i finished my exams"}

try:
    response = requests.post(url, params=params,)
    
    if response.status_code == 200:
        print(response.json())
    else:
        print(f"Error {response.status_code}:")
        print(response.text)

except Exception as e:
    print(f"Connection failed: {e}")

{'Emotion': 'Happy', 'Percentage': 0.5736358761787415}
